# cx_Oracle 모듈
- 파이썬에서 오라클 DBMS와 연동는 다양한 함수를 제공하는 모듈
- Python DB API 2.0 표준을 따른다. 

## 참고
- [cx_Oracle 모듈 홈페이지](https://oracle.github.io/python-cx_Oracle/)
- [Python DB API](https://docs.python.org/ko/3/library/sqlite3.html)

## cx_Oracle 모듈 설치
- [설치 튜토리얼](https://cx-oracle.readthedocs.io/en/latest/user_guide/installation.html)
- 패키지 설치
```
conda install cx_oracle
pip install cx_Oracle --upgrade
```

## 코딩 절차
1. cx_Oracle 모듈 import

2. cx_Oracle모듈의 connect() 함수를 이용해 디비연결
     - ##### DB (접속)

3. Connection 객체의 cursor() 메소드를 이용해 Cursor객체 조회.
    - Cursor객체: sql문을 전송하고 select결과 조회 기능을 제공하는 객체
    - ###### 워크시트라고 생각하자

4. Cursor 의 execute() 메소드를 이용해 SQL문 전송 (임시저장)

5. SELECT 의 경우 결과 조회.
    - 조회결과 한행씩 반환받아 사용 (프라이머리 키같은 애들 조회할때)
    - 조회결과 한번에 반환받아 사용 (여러개를 조회할 때)

6. INSERT/DELETE/UPDATE의 경우  SQL 문 실행 후 Connection의 **commit()(완전저장)** 메소드를 이용해 Transaction 처리

7. Cursor, Connection 연결 닫기

### DB 연결
- connect() 함수를 이용하며 연결 후 Connection 객체를 받는다.
- 연결시 필요한 값
    - **host** ------ 오라클 서버(ip주소)
    - **port번호**--- 오라클 서버
    - **SID[오라클에서는 SID라 부름](나머지는 `DB이름`이라 부름)**
    - **user name**
    - **password**
    - flow : ip => port => SID => user_name, password

### Connection 주요 메소드
- commit(): 커밋
- rollback(): 롤백
- close(): 연결 닫기

In [4]:
# connect()함수를 이용해 연결. 연결(접속) 성공하면 Connection객체를 반환
# Connection객체 : 연결정보를 가지고 있는 객체

# DB연결 1번째 방법
import cx_Oracle
username = 'c##scott_join'
password = 'tiger'
url = 'localhost:1521/XE' # host:post/sid (문자열로)
conn = cx_Oracle.connect(username, password, url) # 연결
print(type(conn))
conn.close() # 닫기

<class 'cx_Oracle.Connection'>


In [5]:
# DB연결 2번째 방법
# username/password@host:port/sid
conn2 = cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE')
print(type(conn))
conn2.close()

<class 'cx_Oracle.Connection'>


#### DSN (Data Source Name) 을 이용한 연결
- DSN: DBMS를 연결하기 위한 정보를 모아 놓은 것. (서로 다른 DBMS의 연결정보를 통일하기 위해서 방식)
- makedsn(host, port번호, SID) 함수 이용해 생성한다.

In [7]:
# DB연결 3번째 방법
username = 'c##scott_join'
password = 'tiger'
host = 'localhost'
port = 1521
sid = 'XE'
dsn = cx_Oracle.makedsn(host, port, sid)
print(dsn)
conn3 = cx_Oracle.connect(username, password, dsn)
print(type(conn3))
conn3.close()

(DESCRIPTION=(ADDRESS=(PROTOCOL=TCP)(HOST=localhost)(PORT=1521))(CONNECT_DATA=(SID=XE)))
<class 'cx_Oracle.Connection'>


In [8]:
# with block을 빠져나오면 자동으로 close()처리.
with cx_Oracle.connect(username, password, dsn) as conn4:
    # DB관련 작업
    pass
print(type(conn4))

<class 'cx_Oracle.Connection'>


## Cursor 객체 
#### (현재위치라는 뜻)
- SQL문을 전송하고 select결과를 조회하는 메소드들을 제공한다.
- Connection객체의 cursor() 메소드로 받아온다.

### Cursor의 주요 메소드
#### SQL 실행 메소드
- execute(sql) : 하나의 sql 문 실행. 많이 사용한다.
- executemany(sql): insert, update, delete 배치 처리
    - insert, update, delete는 다 처리후 conn.commit()으로 커밋 처리해야한다.
    

#### select 결과 조회 메소드
- execute()로 실행한 SQL이 select인 경우 다음 메소드로 조회한다.
- fetchall()  : 조회된 모든 행을 한번에 가져올 때 사용한다. 결과를 tuple들을 묶은 리스트로 반환
- fetchone()  : 호출시 마다 한행씩 반환한다. PK로 조회한 경우 많이 사용한다.
- fetchmany(n): n행만큼 조회한다. n기본값-100. 특정개수반큼 반복문을 이용해 가져올때 사용.

In [9]:
import cx_Oracle
from pprint import pprint # 자료구조를 보기좋게 출력해주는 print함수.

In [18]:
# sql문 : 문자열로 작성. 대소문자 구분 안한다. (keyword of sql 은 보통 대문자로 작성, 컬럼명/테이블명은 소문자)
# sql문 : ';'은 뒤에 붙이지 않는다.
select_sql = "SELECT emp_id, emp_name, job_id FROM emp" # 쿼리문은 오라클에서 한번 실행해보고 넣기

# 오라클에서는 date가 알아서 변환이 되지만 파이썬에서 넘겨줄때는 변환이 안되므로 변환을 해주어야한다.
insert_sql = "INSERT INTO emp VALUES (1500, '홍길동', 'FI_ACCOUNT',\
101, to_date('2021/02/15', 'yyyy/mm/dd'), 30000, 0.2, 100)" # 쿼리문은 똑같이 작성해야한다.
# pk와 uk, fk들이 있는 경우 에러가 난다.
conn = None # Connection을 대입할 변수
cursor = None # Cursor객체를 대입할 변수

# INSERT 후에 SELECT
try:
    #1. 연결
    conn = cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE')
    print('연결완료')
    #2. Cursor생성
    cursor = conn.cursor()
    #3. sql 실행
    cursor.execute(insert_sql)
    cursor.execute(select_sql)
    #4. select결과 조회
    result = cursor.fetchall() # 조회결과를 한번에 반환. 한행 - Tuple, 리스트로 tuple들을 묶어 반환
    print(type(result), type(result[0]))
    print(len(result))
    pprint(result)
    # 5. insert/update/delete를 실행한 경우 commit 처리
    conn.commit()
except Exception as e:
    print(e)
finally:
    # 6. 연결 닫기 : cursor/connection
    if cursor: 
        cursor.close()
    if conn:
        conn.close()

연결완료
<class 'list'> <class 'tuple'>
69
[(100, 'Steven', 'AD_PRES'),
 (101, 'Neena', 'AD_VP'),
 (102, 'Lex', 'AD_VP'),
 (103, 'Alexander', 'IT_PROG'),
 (104, 'Bruce', 'IT_PROG'),
 (105, 'David', 'IT_PROG'),
 (106, 'Valli', 'IT_PROG'),
 (107, 'Diana', 'IT_PROG'),
 (108, 'Nancy', 'FI_MGR'),
 (109, 'Daniel', 'FI_ACCOUNT'),
 (110, 'John', 'FI_ACCOUNT'),
 (111, 'Ismael', 'FI_ACCOUNT'),
 (112, 'Jose Manuel', 'FI_ACCOUNT'),
 (113, 'Luis', 'FI_ACCOUNT'),
 (114, 'Den', 'PU_MAN'),
 (115, 'Alexander', 'PU_MAN'),
 (120, 'Matthew', 'ST_MAN'),
 (121, 'Adam', 'ST_MAN'),
 (122, 'Payam', 'ST_MAN'),
 (123, 'Shanta', 'ST_MAN'),
 (124, 'Kevin', 'ST_MAN'),
 (130, 'Mozhe', None),
 (131, 'James', None),
 (140, 'Joshua', None),
 (145, 'John', 'SA_MAN'),
 (146, 'Karen', 'SA_MAN'),
 (147, 'Alberto', 'SA_MAN'),
 (150, 'Peter', 'SA_REP'),
 (151, 'David', 'SA_REP'),
 (152, 'Peter', 'SA_REP'),
 (153, 'Christopher', 'SA_REP'),
 (154, 'Nanette', 'SA_REP'),
 (155, 'Oliver', 'SA_REP'),
 (156, 'Janette', 'SA_REP'),
 (157

## placeholder(자리표시자)를 이용해 sql 실행
- 파이썬의 format이랑 비슷하게 일한다.
- SQL에 값이 들어갈 자리에 값을 대신할 문자 **`:순번` 또는 `:이름`** 를 넣고 SQL 실행시 값을 전달
> placehold 는 DBMS마다 다르다. 
- execute(SQL, placeholder에 들어갈 값)
    - 딕셔너리: placeholder의 이름-값 형식 `:이름`
    - 리스트나 튜플: placeholder 순서 `:순번`

In [33]:
import cx_Oracle
from pprint import pprint

# SQL문 작성
select_sql = "SELECT * FROM emp WHERE salary BETWEEN :1 AND :2" # format문자열이랑 비슷
insert_sql = "INSERT INTO emp VALUES \
(:id, :name, :job_id, :mgr_id, to_date(:hire_date, 'yyyymmdd'), :salary, :comm_pct, :dept_id)"
# 순번을 사든 이름을 하든 아무거나 상관은지만 개취 or 상황에 맞게

conn = None
cursor = None
try:
    conn = cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE')
    cursor = conn.cursor()
#     SQL 문 실행
#     varchar2/char/nvarchar/nchar - str
#     number (정수/실수) - int/double
#     date - datetime 객체
#     null - None
    param_dict = {
        'id' : 2000,
        'name' : '이순신',
        'job_id' : 'FI_ACCOUNT',
        'mgr_id' : 101,
        'hire_date' : '20210213',
        'salary' : 19990,
        'comm_pct' : None,
        'dept_id' : 110
    }
    cursor.execute(insert_sql, param_dict)

    cursor.execute(select_sql, [10000, 20000])
    result1 = cursor.fetchall()
    print("15000 ~ 20000 사이 salary 조회")
    pprint(result1)
    
    cursor.execute(select_sql, [2000, 3000])
    result2 = cursor.fetchall()
    print("2000 ~ 3000 사이 salary 조회")
    pprint(result2)
    conn.commit()
    
except Exception as e:
    print(e)
finally:
    if cursor:
        cursor.close()
    if conn:
        conn.close()

15000 ~ 20000 사이 salary 조회
[(101,
  'Neena',
  'AD_VP',
  100,
  datetime.datetime(2005, 9, 21, 0, 0),
  17000.0,
  None,
  90),
 (102,
  'Lex',
  'AD_VP',
  100,
  datetime.datetime(2001, 1, 13, 0, 0),
  17000.0,
  None,
  90),
 (104,
  'Bruce',
  'IT_PROG',
  103,
  datetime.datetime(2007, 5, 21, 0, 0),
  18000.0,
  None,
  60),
 (105,
  'David',
  'IT_PROG',
  103,
  datetime.datetime(2005, 6, 25, 0, 0),
  14400.0,
  None,
  60),
 (106,
  'Valli',
  'IT_PROG',
  103,
  datetime.datetime(2006, 2, 5, 0, 0),
  14400.0,
  None,
  60),
 (107,
  'Diana',
  'IT_PROG',
  103,
  datetime.datetime(2007, 2, 7, 0, 0),
  12600.0,
  None,
  60),
 (108,
  'Nancy',
  'FI_MGR',
  101,
  datetime.datetime(2002, 8, 17, 0, 0),
  12008.0,
  None,
  100),
 (113,
  'Luis',
  'FI_ACCOUNT',
  108,
  datetime.datetime(2007, 12, 7, 0, 0),
  10350.0,
  None,
  100),
 (114,
  'Den',
  'PU_MAN',
  100,
  datetime.datetime(2002, 12, 7, 0, 0),
  11000.0,
  None,
  30),
 (145,
  'John',
  'SA_MAN',
  100,
  datetim

## SELECT  결과 조회

### fetchone()
- 조회결과 한행씩 반환한다.
- PK로 조회시 사용한다.(조회결과가 한행은 것)
- 반환 type: tuple

### fetchall()
- 조회한 전체 결과셋을 반환한다.
- 한 row를 Tuple로 묶고 그 Tuple들을 List로 묶어서 반환한다.

### fetchmany(n)
- n 행씩 조회할 때 사용
- n 기본값 100
- 지정값만큼 끈어서 가져온다.paging 개념

In [37]:
# emp_id(PK)로 직원 정보를 조회하는 함수
def select_emp_by_id(emp_id):
    sql = "SELECT * FROM emp WHERE emp_id = :1"
#     conn, cursor = None, None
    with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
        with conn.cursor() as cursor:
            # sql 실행
            cursor.execute(sql, [emp_id])
            result = cursor.fetchone() # 1행 조회, 조회결과가 없으면 None을 반환
            return result

In [40]:
r = select_emp_by_id(100)
if r:
    print(r, "id:", r[0], '이름: ', r[1])
else:
    print('조회결과 없습니다.')

(100, 'Steven', 'AD_PRES', 100, datetime.datetime(2003, 6, 17, 0, 0), 27000.0, 0.2, 90) id: 100 이름:  Steven


In [42]:
r = select_emp_by_id(10000)
if r:
    print(r, "id:", r[0], '이름: ', r[1])
else:
    print('조회결과 없습니다.', r)

조회결과 없습니다. None


In [50]:
sql = "SELECT * FROM emp ORDER BY emp_id"
with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchmany(10) # 10개만 조회
        print(len(result))
        print(result)
        
        result = cursor.fetchmany(10) # 다음 10개
        print(len(result))
        print(result)
        
        result = cursor.fetchmany(10)
        print(len(result))
        print(result)

10
[(100, 'Steven', 'AD_PRES', 100, datetime.datetime(2003, 6, 17, 0, 0), 27000.0, 0.2, 90), (101, 'Neena', 'AD_VP', 100, datetime.datetime(2005, 9, 21, 0, 0), 17000.0, None, 90), (102, 'Lex', 'AD_VP', 100, datetime.datetime(2001, 1, 13, 0, 0), 17000.0, None, 90), (103, 'Alexander', 'IT_PROG', 102, datetime.datetime(2006, 1, 3, 0, 0), 27000.0, None, 60), (104, 'Bruce', 'IT_PROG', 103, datetime.datetime(2007, 5, 21, 0, 0), 18000.0, None, 60), (105, 'David', 'IT_PROG', 103, datetime.datetime(2005, 6, 25, 0, 0), 14400.0, None, 60), (106, 'Valli', 'IT_PROG', 103, datetime.datetime(2006, 2, 5, 0, 0), 14400.0, None, 60), (107, 'Diana', 'IT_PROG', 103, datetime.datetime(2007, 2, 7, 0, 0), 12600.0, None, 60), (108, 'Nancy', 'FI_MGR', 101, datetime.datetime(2002, 8, 17, 0, 0), 12008.0, None, 100), (109, 'Daniel', 'FI_ACCOUNT', 108, datetime.datetime(2002, 8, 16, 0, 0), 9000.0, None, 100)]
10
[(110, 'John', 'FI_ACCOUNT', 108, datetime.datetime(2005, 9, 28, 0, 0), 8200.0, None, 100), (111, 'Ismae

### cursor를 for in 문을 이용해 select결과 조회
- select후 cursor는 Iterable 한 객체.
- cursor는 반복시마다 조회한 한행을 반환.

In [55]:
sql = "SELECT * FROM emp WHERE comm_pct IS NOT NULL ORDER BY emp_id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        # 한행씩 처리하는 겨우
        for idx, row in enumerate(cursor, 1): # 한번 반복할 때 마다 cursor.fetchone()을 실행.
            print(f'{idx}. {row}')
            
#         r = cursor.fetchone()
#         while r:
#             print(r)
#             r = cursor.fetchone()

1. (100, 'Steven', 'AD_PRES', 100, datetime.datetime(2003, 6, 17, 0, 0), 27000.0, 0.2, 90)
2. (151, 'David', 'SA_REP', 145, datetime.datetime(2005, 3, 24, 0, 0), 9500.0, 0.25, 80)
3. (152, 'Peter', 'SA_REP', 145, datetime.datetime(2005, 8, 20, 0, 0), 9000.0, 0.25, 80)
4. (153, 'Christopher', 'SA_REP', 145, datetime.datetime(2006, 3, 30, 0, 0), 8000.0, 0.2, 80)
5. (154, 'Nanette', 'SA_REP', 145, datetime.datetime(2006, 12, 9, 0, 0), 7500.0, 0.2, 80)
6. (155, 'Oliver', 'SA_REP', 145, datetime.datetime(2007, 11, 23, 0, 0), 10500.0, 0.15, 80)
7. (161, 'Sarath', 'SA_REP', 146, datetime.datetime(2006, 11, 3, 0, 0), 10500.0, 0.25, 80)
8. (162, 'Clara', 'SA_REP', 147, datetime.datetime(2005, 11, 11, 0, 0), 10500.0, 0.25, 80)
9. (163, 'Danielle', 'SA_REP', 147, datetime.datetime(2007, 3, 19, 0, 0), 9500.0, 0.15, 80)
10. (164, 'Mattea', 'SA_REP', 147, datetime.datetime(2008, 1, 24, 0, 0), 7200.0, 0.1, 80)
11. (165, 'David', 'SA_REP', 147, datetime.datetime(2008, 2, 23, 0, 0), 10200.0, 0.1, 80)
1

In [58]:
sql = "SELECT job_id, sum(salary), round(avg(salary), 2) FROM emp GROUP BY job_id"
with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
        pprint(result)

[('AD_VP', 34000, 17000),
 ('FI_ACCOUNT', 121540, 12154),
 ('HR_REP', 9750, 9750),
 ('PU_MAN', 20100, 10050),
 ('AC_MGR', 12008, 12008),
 ('AD_ASST', 8250, 8250),
 ('IT_PROG', 86400, 17280),
 (None, 24300, 4050),
 ('SA_MAN', 39500, 13166.67),
 ('AC_ACCOUNT', 8300, 8300),
 ('FI_MGR', 12008, 12008),
 ('ST_MAN', 42550, 8510),
 ('AD_PRES', 27000, 27000),
 ('MK_MAN', 13000, 13000),
 ('SA_REP', 257550, 9198.21),
 ('MK_REP', 9000, 9000),
 ('PR_REP', 10000, 10000)]


In [60]:
sql = "SELECT e.emp_name, d.dept_name FROM emp e left join dept d ON e.dept_id = d.dept_id"
with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
    with conn.cursor() as cursor:
        cursor.execute(sql)
        result = cursor.fetchall()
        pprint(result)

[('Jennifer', 'Administration'),
 ('박영희', 'Administration'),
 ('박영희', 'Administration'),
 ('Michael', 'Marketing'),
 ('Pat', 'Marketing'),
 ('홍길동', 'Marketing'),
 ('Den', 'Purchasing'),
 ('Alexander', 'Purchasing'),
 ('Susan', 'Human Resources'),
 ('Matthew', 'Shipping'),
 ('Adam', 'Shipping'),
 ('Payam', 'Shipping'),
 ('Shanta', 'Shipping'),
 ('Kevin', 'Shipping'),
 ('Mozhe', 'Shipping'),
 ('James', 'Shipping'),
 ('Joshua', 'Shipping'),
 ('Anthony', 'Shipping'),
 ('Timothy', 'Shipping'),
 ('Randall', 'Shipping'),
 ('Alexander', 'IT'),
 ('Bruce', 'IT'),
 ('David', 'IT'),
 ('Valli', 'IT'),
 ('Diana', 'IT'),
 ('Hermann', 'Public Relations'),
 ('John', 'Sales'),
 ('Karen', 'Sales'),
 ('Alberto', 'Sales'),
 ('Peter', 'Sales'),
 ('David', 'Sales'),
 ('Peter', 'Sales'),
 ('Christopher', 'Sales'),
 ('Nanette', 'Sales'),
 ('Oliver', 'Sales'),
 ('Janette', 'Sales'),
 ('Patrick', 'Sales'),
 ('Allan', 'Sales'),
 ('Lindsey', 'Sales'),
 ('Louise', 'Sales'),
 ('Sarath', 'Sales'),
 ('Clara', 'Sales')

## INSERT / UPDATE/ DELETE
- SQL 실행 후 Connection객체의 commit()을 호출 해야 최종적으로 적용된다.
- commit()
    - 최종 적용
- rollback()
    - 변경전 상태로 되돌리기

### 사용할 테이블
```sql
create table test(
    id varchar2(10),
    name varchar2(100),
    age number(3));
```

In [119]:
create_sql = "create table test (id varchar2(10), name varchar2(100), age number(3))"
insert_sql = "INSERT INTO test VALUES (:1, :2, :3)"

# insert_sql = "INSERT INTO emp VALUES (1500, '홍길동', 'FI_ACCOUNT',\
# 101, to_date('2021/02/15', 'yyyy/mm/dd'), 30000, 0.2, 100)"


with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
#         try:
#             cursor.execute(create_sql) # 테이블 생성
#         except Exception as e:
#             print(e)
        cursor.execute(insert_sql, [1, '이름1', 20])
        cursor.execute(insert_sql, [2, '이름2', 30])
        cursor.execute(insert_sql, [3, '이름3', 40])

In [113]:
delete_sql = "DELETE FROM test WHERE name = :1"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(delete_sql, ['이름1'])
        conn.commit()

In [114]:
update_sql = "UPDATE test SET name = :1"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.execute(update_sql, ['새이름'])
        conn.commit()

In [120]:
select_sql = "SELECT * FROM test order by id"
with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
            cursor.execute(select_sql) 
            pprint(cursor.fetchall())

[('2', '새이름', 30),
 ('3', '새이름', 40),
 ('a-2', '새이름', 30),
 ('a-2', '새이름', 30),
 ('a-3', '새이름', 35),
 ('a-3', '새이름', 35),
 ('a-4', '새이름', 40),
 ('a-4', '새이름', 40)]


## batch(일괄작업) 처리
- 한번에 메소드 호출로 다수 행을 처리한다.
- executemany(sql, placeholder에 전달할값) - 주로 인설트 할때 많이 쓴다.
    - placeholder에 전달할 값을 list로 묶어서 전달하면 sql문이 그 개수만큼 한번에 처리된다.

In [95]:
insert_sql = 'INSERT INTO test VALUES (:1, :2, :3)'
param = [
    ['a-1', '이름1', 20],
    ['a-2', '이름2', 30],
    ['a-3', '이름3', 35],
    ['a-4', '이름4', 40]
]

with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
    with conn.cursor() as cursor:
        cursor.executemany(insert_sql, param) # executemany가 for문 역할을 대신해준다.
        conn.commit()

## TODO : 매개변수로 dept_id를 받아서 EMP 테이블에서 그 부서의 직원들의 salary를 두배 update하는 함수.

In [1]:
# import cx_Oracle
# from pprint import pprint
# def update_salary_by_dept_id(dept_id):
#     update_sql = "UPDATE emp SET salary = salary*2 where dept_id = :1"
#     with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
#         with conn.cursor() as cursor:
#             cursor.execute(update_sql, [dept_id])
#             conn.commit()
  
import cx_Oracle
def update_salary_by_dept_id(dept_id):
    sql = "UPDATE emp SET salary = salary *2 where dept_id = :1"
    with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn: # DB연결
        with conn.cursor() as cursor: # 커서 생성
            cursor.execute(sql, [dept_id]) # update문 실행
            conn.commit()

In [2]:
update_salary_by_dept_id(80)

## TODO : 매개변수로 job_id를 받아서 그 업무를 하는 직원들을 EMP 테이블에서 삭제하는 함수.

In [144]:
def delete_job_id_by_emp(job_id):
    delete_sql = "delete from emp where job_id = :1"
    with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
        with conn.cursor() as cursor:
            cursor.execute(delete_sql, [job_id])

In [3]:
def delete_emp_by_job_id(job_id):
    # null값이 있을 경우 조건문으로 다룬다.
    if job_id != None:        
        sql = "delete from emp where job_id = :1"
    else:
        sql = "delete from emp where job_id is null"
    with cx_Oracle.connet("c##scott_join/tiger@localhost:1521/XE") as conn:
        with conn.cursor() as cursor:
            if job_id != None:
                cursor.execute(sql, [job_id])
            else:
                cursor.execute(sql)
            cursor.commit()

In [ ]:
delete_emp_by_job_id(None) # job id 가 null인 행을 삭제는 그냥 안된다.
## delete from emp where job_id is null로 지워야 된다.

# TODO : 매개변수로 직원 이름을 입력받아 EMP 테이블에서 그  직원들의 전체 정보를 조회결과를 출력하는 함수.

In [4]:
def select_all_by_emp(emp_name):
    select_sql = "select * from emp where emp_name = :1"
    with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/Xe") as conn:
        with conn.cursor() as cursor:
            cursor.execute(select_sql, [emp_name])
            result = cursor.fetchall()
            return result

In [11]:
import cx_Oracle
def select_emp_by_name(emp_name):
#     sql = 'select * from emp where emp_name = :1'
    sql = "select * from emp where emp_name like '%'||:1||'%'" # like연산자의 placeholder사용할때 조심
#     sql = "select * from emp where emp_name like '%'||:name||'%'"
    with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, [emp_name])
#             cursor.execute(sql, {"name": emp_name})
            result = cursor.fetchall()
            return result
print(select_emp_by_name('David'))

[(105, 'David', 'IT_PROG', 103, datetime.datetime(2005, 6, 25, 0, 0), 14400.0, None, 60), (151, 'David', 'SA_REP', 145, datetime.datetime(2005, 3, 24, 0, 0), 19000.0, 0.25, 80), (165, 'David', 'SA_REP', 147, datetime.datetime(2008, 2, 23, 0, 0), 20400.0, 0.1, 80)]


# TODO: 사용자로 부터 직원정보를 입력받아 EMP 테이블에 입력한다.
- emp_id, emp_name, job_id, mgr_id, hire_date, salary, comm_pct, dept_id 를 하나씩 입력받는다.
- job_id, mgr_id, comm_pct, dept_id 을 입력하지 않은 경우 NULL 을 넣는다. (placehold 사용시 None을 대입)
- 입력된 값을 EMP 테이블에 insert 한다.
- 주의 job_id, mgr_id, dept_id는 FK 컬럼이므로 부모테이블의 PK값을 넣어야 한다.

In [ ]:
import cx_Oracle

def insert_info():
    insert_sql = 'INSERT INTO emp VALUES (:emp_id, :emp_name, :job_id, :mgr_id, :hire_date, :salary, :comm_pct, :dept_id)'
    param = {
        "emp_id" : int(input("emp_id 입력")),
        "emp_name" : input("emp_name 입력"),
        "job_id" : input("job_id 입력"), # null 허용
        "mgr_id" : input("mgr_id 입력"), # null 허용
        'hire_date' : input("hire_date 입력"),
        'salary' : input("salary 입력"),
        'comm_pct' : input("comm_pct 입력"), # null 허용
        'dept_id' : input("dept_id 입력") # null 허용
    }
    with cx_Oracle.connect('c##scott_join/tiger@localhost:1521/XE') as conn:
        with conn.cursor() as cursor:
            cursor.executemany(insert_sql, param)
            conn.commit()


In [23]:
import cx_Oracle

def insert_emp():
    # 사용자로 부터 insert할 값들을 입력 받기 + 검증
    emp_id = int(input("직원ID: "))
    
    emp_name = input("이름: ")
    
    job_id = input("업무ID: ")
    if len(job_id.strip()) == 0:
        job_id = None
        
    try:
        mgr_id = int(input("상사ID: "))
    except:
        mgr_id = None
        
    hiredate = input("입사일(형식:yyyymmdd) :")
    salary = int(input('급여: '))
    
    try:
        comm_pct = float(input('커미션률:'))
        if comm_pct >= 1:
            raise Exception()
    except:
        comm_pct = None
        
    try:
        dept_id = input("부서ID: ")
    except:
        dept_id = None
        
    param = {
        "emp_id":emp_id,
        "emp_name":emp_name,
        "job_id" : job_id,
        "mgr_id" : mgr_id,
        "hire_date" : hire_date,
        "salary":salary,
        "comm_pct":comm_pct,
        "dept_id":dept_id
    }
    sql = "INSERT INTO emp VALUES (:emp_id, :emp_name, :job_id, :mgr_id, to_date(:hire_date, 'yyyymmdd'), :salary, :comm_pct, :dept_id)"
    with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
        with conn.cursor() as cursor:
            cursor.execute(sql, param)
            conn.commit()
            print("insert완료") 

In [24]:
insert_emp()

직원ID: 3400
이름: 윤우상
업무ID: FI_ACCOUNT
상사ID: 
입사일(형식:yyyymmdd) :19960430
급여: 3000
커미션률:
부서ID: 
insert완료


In [25]:
"""
--TODO: 부서별 직원의 급여에 대한 통계 테이블 생성. 
--      조회결과를 insert. 집계: 합계, 평균, 최대, 최소, 분산, 표준편차
create table salary_stat(
    dept_id number(6),
    salary_sum number(15,2),
    salary_avg number(10, 2),
    salary_max number(7,2),
    salary_min number(7,2),
    salary_var number(20,2),
    salary_stddev number(7,2)
);
"""

'\n--TODO: 부서별 직원의 급여에 대한 통계 테이블 생성. \n--      조회결과를 insert. 집계: 합계, 평균, 최대, 최소, 분산, 표준편차\ncreate table salary_stat(\n    dept_id number(6),\n    salary_sum number(15,2),\n    salary_avg number(10, 2),\n    salary_max number(7,2),\n    salary_min number(7,2),\n    salary_var number(20,2),\n    salary_stddev number(7,2)\n);\n'

In [8]:
import cx_Oracle

# create_sql = "CREATE TABLE salary_stat2\
# (dept_id number(6), salary_sum number(15,2), salary_avg numer(10, 2), salary_max number(7, 2),\
# salary_min number(7, 2), salary_var number(20, 2), salary_stddev number(7, 2))"

insert_sql = "INSERT INTO salary_stat2 \
SELECT :dept_id, :salary_sum, :salary_avg, :salary_max, :salary_min, :salary_var, :salary_stddev\
FROM emp GROUP BY dept_id"

# insert_sql = "INSERT INTO salary_stat2 SELECT dept_id, sum(salary), round(avg(salary), 2),\
# max(salary), min(salary), round(variance(salary), 2), round(stddev(salary), 2) FROM emp GROUP BY dept_id"
with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
    with conn.cursor() as cursor:
        param = {
            "dept_id" : dept_id,
            "salary_sum" : sum(salary),
            "salary_avg" : round(avg(salary), 2),
            "salary_max" : max(salary),
            "salary_min" : min(salary),
            "salary_var" : round(variance(salary), 2),
            "salary_stddev": round(stddev(salary), 2)
        }
        cursor.execute(insert_sql, param)
        conn.commit()

NameError: name 'dept_id' is not defined

In [5]:
import cx_Oracle

# create_sql = "CREATE TABLE salary_stat1\
# (dept_id number(6), salary_sum number(15,2), salary_avg number(10, 2), salary_max number(7, 2),\
# salary_min number(7, 2), salary_var number(20, 2), salary_stddev number(7, 2))"

insert_sql = "INSERT INTO salary_stat SELECT dept_id, sum(salary), round(avg(salary), 2),\
max(salary), min(salary), round(variance(salary), 2), round(stddev(salary), 2) FROM emp GROUP BY dept_id"

with cx_Oracle.connect("c##scott_join/tiger@localhost:1521/XE") as conn:
    with conn.cursor() as cursor:
#         cursor.execute(create_sql)
        cursor.execute(insert_sql)
        conn.commit()